In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pywikibot as pwb
import os
from bs4 import BeautifulSoup
import meerkat as mk
from meerkat.interactive.formatter import WebsiteFormatter

os.environ["AI21_API_KEY"] = "ws61J6akyMa0RuiuJW117Um8a5xrKUZ3"

In [3]:
mk.gui.start()

Trying to find an open port in (7860, 7960).

Found open port: 7861

Trying to find an open port in (7862, 7962).

Found open port: 7863

Trying port: 7863

Meerkat interactive mode started! API docs on http://127.0.0.1:7861/docs and GUI server on http://localhost:7863.

NetworkInfo(api=<fastapi.applications.FastAPI object at 0x162be34c0>, api_server_port=7861, api_server=<meerkat.interactive.server.Server object at 0x10609b850>, npm_server_port=7863, npm_process=<Popen: returncode: None args: ['npm', 'run', 'dev', '--', '--port', '7863',...>, api_server_name='127.0.0.1', npm_server_name='localhost', shareable_npm_server_name=None, shareable_api_server_name=None, npm_out_path='/var/folders/0g/9gpzpkwn2j74ntfk90_9t31m0000gp/T/tmp60ale4q4.out', npm_err_path='/var/folders/0g/9gpzpkwn2j74ntfk90_9t31m0000gp/T/tmpmm2xzlme.err')

In [23]:

import requests
import pywikibot as pwb
import os
import meerkat as mk
from meerkat.interactive.formatter import WebsiteFormatter

url = "https://query.wikidata.org/sparql"
query = """
SELECT ?country ?countryLabel ?article
WHERE {
  ?country wdt:P31 wd:Q6256.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
LIMIT 10
"""
r = requests.get(url, params={"format": "json", "query": query})
data = r.json()


df = mk.DataFrame(
    [
        {"uri": row["country"]["value"], "country": row["countryLabel"]["value"]}
        for row in data["results"]["bindings"]
    ]
)
df["uri"].formatter = WebsiteFormatter()
df["qid"] = df["uri"].map(lambda x: os.path.basename(x))

def get_wikipedia_title(qid: str):
    site = pwb.Site("wikipedia:en")
    repo = site.data_repository()
    item = pwb.ItemPage(repo, qid)
    title = item.sitelinks["enwiki"].title
    return title

df["title"] = df.map(get_wikipedia_title, pbar=True)
df["wikipedia_url"] = df.map(
    lambda title: f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"
)
df["wikipedia_url"].formatter = WebsiteFormatter()


100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


In [5]:
def get_wikipedia_html(title: str):
    return requests.get(f"http://en.wikipedia.org/w/api.php?action=parse&format=json&page={title}&prop=text").json()["parse"]["text"]["*"]


df["html"] = df.defer(get_wikipedia_html, outputs="single")

In [21]:
df

,country,uri,qid,title,wikipedia_url,html,prompt
0,Canada,http://www.wikidata.org/entity/Q16,Q16,Canada,https://en.wikipedia.org/wiki/Canada,DeferredCell,"\nIn 2011, Canadian forces participated in the NATO-led intervention into the Libyan Civil War,[118] and also became involved in battling the Islamic State insurgency in Iraq in the mid-2010s.[119] The COVID-19 pandemic in Canada began on January 27, 2020, with wide social and economic disruption.[120] In 2021, the remains of hundreds of Indigenous people were discovered near the former sites of Canadian Indian residential schools.[121] Administered by the Canadian Catholic Church and funded by the Canadian government from 1828 to 1997, these boarding schools attempted to assimilate Indigenous children into Euro-Canadian culture.[122]\n\nIn common with many other developed countries, Canada is experiencing an increase in healthcare expenditures due to a demographic shift toward an older population, with more retirees and fewer people of working age. In 2006, the average age in Canada was 39.5 years;[317] it rose to 42.4 years by 2018[318] before falling slightly to 41.9 in 2021.[309] Life expectancy is 81.1 years.[319] A 2016 report by the chief public health officer found that 88 percent of Canadians, one of the highest proportions of the population among G7 countries, indicated that they ""had good or very good health"".[320] Eighty percent of Canadian adults self-report having at least one major risk factor for chronic disease: smoking, physical inactivity, unhealthy eating or excessive alcohol use.[321] Canada has one of the highest rates of adult obesity among Organisation for Economic Co-operation and Development (OECD) countries, contributing to approximately 2.7 million cases of diabetes (types 1 and 2 combined).[321] Four chronic diseases—cancer (leading cause of death), cardiovascular diseases, respiratory diseases and diabetes—account for 65 percent of deaths in Canada.[322][323]\n\nBased on the prior context, Has Canada experienced any famines?\nYes or no answer:"
1,Japan,http://www.wikidata.org/entity/Q17,Q17,Japan,https://en.wikipedia.org/wiki/Japan,DeferredCell,"\nJapan spent 10.74% of its total GDP on healthcare in 2019.[267] In 2020, the overall life expectancy in Japan at birth was 84.62 years (81.64 years for males and 87.74 years for females), the highest in the world;[268] while it had a very low infant mortality rate (2 per 1,000 live births).[269] Since 1981, the principal cause of death in Japan is cancer, which accounted for 27% of the total deaths in 2018—followed by cardiovascular diseases, which led to 15% of the deaths.[270] Japan has one of the world's highest suicide rates, which is considered a major social issue.[271] Another significant public health issue is smoking among Japanese men.[272] However, Japan has the lowest rate of heart disease in the OECD, and the lowest level of dementia among developed countries.[273]\n\nJapan is substantially prone to earthquakes, tsunami and volcanic eruptions because of its location along the Pacific Ring of Fire.[85] It has the 17th highest natural disaster risk as measured in the 2016 World Risk Index.[86] Japan has 111 active volcanoes.[87] Destructive earthquakes, often resulting in tsunami, occur several times each century;[88] the 1923 Tokyo earthquake killed over 140,000 people.[89] More recent major quakes are the 1995 Great Hanshin earthquake and the 2011 Tōhoku earthquake, which triggered a large tsunami.[70]\n\nBased on the prior context, Has Japan experienced any famines?\nYes or no answer:"
2,Norway,http://www.wikidata.org/entity/Q20,Q20,Norway,https://en.wikipedia.org/wiki/Norway,DeferredCell,"\nThe famine of 1695–1696 killed roughly 10% of Norway's population.[80] The harvest failed in Scandinavia at least nine times between 1740 and 1800, with great loss of life.[81]\n\nIn 1349, the Black Death radically altered Norway, killing between 50% and 60% of its population[75] and leaving it in a

In [6]:
df.gui.gallery(main_column="wikipedia_url", tag_columns=["title"])

In [8]:
from typing import Dict, List


def get_paragraphs(html: str, title: str) -> List[Dict]:
    soup = BeautifulSoup(html)

    # Extract the headers
    headers = soup.find_all(["h1","h2","h3","h4","h5","h6"])

    # Create a dictionary to store the headers and their corresponding sections
    paragraphs = []

    # Iterate through the headers
    for header in headers:
        level = int(header.name[1])
        section = header.find_next_sibling()
        section_text = []
        while section and section.name not in ["h1","h2","h3","h4","h5","h6"]:
            if section.name == "p":
                paragraphs.append({
                    "section": header.get_text(),
                    "paragraph": section.get_text(),
                    "title": title
                })
            section = section.find_next_sibling()
    return paragraphs

import itertools
ctx_df = mk.DataFrame(list(itertools.chain(*df.map(get_paragraphs, outputs="single").data)))


In [9]:
ctx_df

,paragraph,section,title
0,"While a variety of theories have been postulated for the etymological origins of Canada, the name is now accepted as coming from the St. Lawrence Iroquoian word kanata, meaning ""village"" or ""settlement"".[9] In 1535, Indigenous inhabitants of the present-day Quebec City region used the word to direct French explorer Jacques Cartier to the village of Stadacona.[10] Cartier later used the word Canada to refer not only to that particular village but to the entire area subject to Donnacona (the chief at Stadacona);[10] by 1545, European books and maps had begun referring to this small region along the Saint Lawrence River as Canada.[10]\n",Etymology[edit],Canada
1,"From the 16th to the early 18th century, ""Canada"" referred to the part of New France that lay along the Saint Lawrence River.[11] In 1791, the area became two British colonies called Upper Canada and Lower Canada. These two colonies were collectively named the Canadas until their union as the British Province of Canada in 1841.[12]\n",Etymology[edit],Canada
2,"Upon Confederation in 1867, Canada was adopted as the legal name for the new country at the London Conference, and the word Dominion was conferred as the country's title.[13] By the 1950s, the term Dominion of Canada was no longer used by the United Kingdom, which considered Canada a ""Realm of the Commonwealth"".[14] The government of Louis St. Laurent ended the practice of using Dominion in the statutes of Canada in 1951.[15][16][17]\n",Etymology[edit],Canada
3,"The Canada Act 1982, which brought the constitution of Canada fully under Canadian control, referred only to Canada. Later that year, the name of the national holiday was changed from Dominion Day to Canada Day.[18] The term Dominion was used to distinguish the federal government from the provinces, though after the Second World War the term federal had replaced dominion.[19]\n",Etymology[edit],Canada
4,"Indigenous peoples in present-day Canada include the First Nations, Inuit, and Métis,[20] the last being of mixed descent who originated in the mid-17th century when First Nations people married European settlers and subsequently developed their own identity.[20]\n",Indigenous peoples[edit],Canada
...,...,...,...
1355,"The open sandwiches on rye bread, known as smørrebrød, which in their basic form are the usual fare for lunch, can be considered a national speciality when prepared and decorated with a variety of fine ingredients. Hot meals traditionally consist of ground meats, such as frikadeller (meat balls of veal and pork) and hakkebøf (minced beef patties), or of more substantial meat and fish dishes such as flæskesteg (roast pork with crackling) and kogt torsk (poached cod) with mustard sauce and trimmings. Denmark is known for its Carlsberg and Tuborg beers and for its akvavit and bitters.\n",Cuisine[edit],Denmark
1356,"Since around 1970, chefs and restaurants across Denmark have introduced gourmet cooking, largely influenced by French cuisine. Also inspired by continental practices, Danish chefs have recently developed a new innovative cuisine and a series of gourmet dishes based on high-quality local produce known as New Danish cuisine.[263] As a result of these developments, Denmark now has a considerable number of internationally acclaimed restaurants of which several have been awarded Michelin stars. This includes Geranium and Noma in Copenhagen.\n",Cuisine[edit],Denmark
1357,"Sports are popular in Denmark, and its citizens participate in and watch a wide variety. The national sport is football, with over 320,000 players in more than 1600 clubs.[264] Denmark qualified six times consecutively for the European Championships between 1984 and 2004, and were crowned European champions in 1992; other significant achievements include winning the Confederations Cup in 1995 and reaching the quarter-final of the 1998 World Cup. Notable Danish footballers include Allan Simonsen, named the best player in Europe in 1977, P

In [10]:
from typing import Sequence
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer

context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

[2023-01-16 11:35:50,002][WARNING][torch.distributed.elastic.multiprocessing.redirects:27] :: NOTE: Redirects are currently not supported in Windows or MacOs.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from

In [11]:
import torch


@torch.no_grad()
def encode_context(text: Sequence[str]):
    return context_encoder(context_tokenizer(list(text), return_tensors="pt", padding=True)["input_ids"]).pooler_output

@torch.no_grad()
def encode_question(text: Sequence[str]):
    return question_encoder(question_tokenizer(list(text), return_tensors="pt", padding=True)["input_ids"]).pooler_output

ctx_df["ctx_emb"] = ctx_df["paragraph"].map(encode_context, is_batched_fn=True, batch_size=4)

In [12]:
ctx_df

,paragraph,section,title,ctx_emb
0,"While a variety of theories have been postulated for the etymological origins of Canada, the name is now accepted as coming from the St. Lawrence Iroquoian word kanata, meaning ""village"" or ""settlement"".[9] In 1535, Indigenous inhabitants of the present-day Quebec City region used the word to direct French explorer Jacques Cartier to the village of Stadacona.[10] Cartier later used the word Canada to refer not only to that particular village but to the entire area subject to Donnacona (the chief at Stadacona);[10] by 1545, European books and maps had begun referring to this small region along the Saint Lawrence River as Canada.[10]\n",Etymology[edit],Canada,torch.Tensor(shape=torch.Size([768]))
1,"From the 16th to the early 18th century, ""Canada"" referred to the part of New France that lay along the Saint Lawrence River.[11] In 1791, the area became two British colonies called Upper Canada and Lower Canada. These two colonies were collectively named the Canadas until their union as the British Province of Canada in 1841.[12]\n",Etymology[edit],Canada,torch.Tensor(shape=torch.Size([768]))
2,"Upon Confederation in 1867, Canada was adopted as the legal name for the new country at the London Conference, and the word Dominion was conferred as the country's title.[13] By the 1950s, the term Dominion of Canada was no longer used by the United Kingdom, which considered Canada a ""Realm of the Commonwealth"".[14] The government of Louis St. Laurent ended the practice of using Dominion in the statutes of Canada in 1951.[15][16][17]\n",Etymology[edit],Canada,torch.Tensor(shape=torch.Size([768]))
3,"The Canada Act 1982, which brought the constitution of Canada fully under Canadian control, referred only to Canada. Later that year, the name of the national holiday was changed from Dominion Day to Canada Day.[18] The term Dominion was used to distinguish the federal government from the provinces, though after the Second World War the term federal had replaced dominion.[19]\n",Etymology[edit],Canada,torch.Tensor(shape=torch.Size([768]))
4,"Indigenous peoples in present-day Canada include the First Nations, Inuit, and Métis,[20] the last being of mixed descent who originated in the mid-17th century when First Nations people married European settlers and subsequently developed their own identity.[20]\n",Indigenous peoples[edit],Canada,torch.Tensor(shape=torch.Size([768]))
...,...,...,...,...
1355,"The open sandwiches on rye bread, known as smørrebrød, which in their basic form are the usual fare for lunch, can be considered a national speciality when prepared and decorated with a variety of fine ingredients. Hot meals traditionally consist of ground meats, such as frikadeller (meat balls of veal and pork) and hakkebøf (minced beef patties), or of more substantial meat and fish dishes such as flæskesteg (roast pork with crackling) and kogt torsk (poached cod) with mustard sauce and trimmings. Denmark is known for its Carlsberg and Tuborg beers and for its akvavit and bitters.\n",Cuisine[edit],Denmark,torch.Tensor(shape=torch.Size([768]))
1356,"Since around 1970, chefs and restaurants across Denmark have introduced gourmet cooking, largely influenced by French cuisine. Also inspired by continental practices, Danish chefs have recently developed a new innovative cuisine and a series of gourmet dishes based on high-quality local produce known as New Danish cuisine.[263] As a result of these developments, Denmark now has a considerable number of internationally acclaimed restaurants of which several have been awarded Michelin stars. This includes Geranium and Noma in Copenhagen.\n",Cuisine[edit],Denmark,torch.Tensor(shape=torch.Size([768]))
1357,"Sports are popular in Denmark, and its citizens participate in and watch a wide variety. The national sport is football, with over 320,000 players in more than 1600 clubs.[264] Denmark qualified six times consecutively for the European Championships between 1984 an

In [13]:
def get_prompt(question: str, country: str):
    question = question.format(country=country)
    question_emb = encode_question([question])
    ctx = "\n".join(mk.search(data=ctx_df, query=question_emb, by="ctx_emb", k=2)["paragraph"])
    return f"""
{ctx}
Based on the prior context, {question}
Yes or no answer:"""

from functools import partial
df["prompt"] = df.map(partial(get_prompt, "Has {country} experienced any famines?"))


In [ ]:
df

In [31]:
df["answer"] = "unanswered"

In [15]:
print(df["prompt"][7])


The Finnish famine of 1866–1868 occurred after freezing temperatures in early September ravaged crops,[64] and it killed approximately 15% of the population, making it one of the worst famines in European history. The famine led the Russian Empire to ease financial regulations, and investment rose in the following decades. Economic and political development was rapid.[65] The gross domestic product (GDP) per capita was still half of that of the United States and a third of that of Britain.[65]

The famine of 1695–1696 killed roughly 10% of Norway's population.[80] The harvest failed in Scandinavia at least nine times between 1740 and 1800, with great loss of life.[81]

Based on the prior context, Has Finland experienced any famines?
Yes or no answer:


In [32]:
df["answer"][0] = "no"
df["answer"][1] = "no"
df["answer"][7] = "yes"

In [33]:
train_df = df[df["answer"] != "unanswered"] 
examples = "\n".join(train_df["prompt"] + " " + train_df["answer"]) + "\n"


In [19]:
from manifest import Manifest

# Start a manifest session to OpenAI - default `engine=text-davinci-002`
manifest = Manifest(
    client_name="ai21",
    cache_name="sqlite",
    session_id=None,
    engine="j1-jumbo",
    max_tokens="10",
    temperature=.1
)
print(manifest.run(df["prompt"][2]))

A:

Yes, Norway has experienced


In [20]:
df

,country,uri,qid,title,wikipedia_url,html,prompt
0,Canada,http://www.wikidata.org/entity/Q16,Q16,Canada,https://en.wikipedia.org/wiki/Canada,DeferredCell,"\nIn 2011, Canadian forces participated in the NATO-led intervention into the Libyan Civil War,[118] and also became involved in battling the Islamic State insurgency in Iraq in the mid-2010s.[119] The COVID-19 pandemic in Canada began on January 27, 2020, with wide social and economic disruption.[120] In 2021, the remains of hundreds of Indigenous people were discovered near the former sites of Canadian Indian residential schools.[121] Administered by the Canadian Catholic Church and funded by the Canadian government from 1828 to 1997, these boarding schools attempted to assimilate Indigenous children into Euro-Canadian culture.[122]\n\nIn common with many other developed countries, Canada is experiencing an increase in healthcare expenditures due to a demographic shift toward an older population, with more retirees and fewer people of working age. In 2006, the average age in Canada was 39.5 years;[317] it rose to 42.4 years by 2018[318] before falling slightly to 41.9 in 2021.[309] Life expectancy is 81.1 years.[319] A 2016 report by the chief public health officer found that 88 percent of Canadians, one of the highest proportions of the population among G7 countries, indicated that they ""had good or very good health"".[320] Eighty percent of Canadian adults self-report having at least one major risk factor for chronic disease: smoking, physical inactivity, unhealthy eating or excessive alcohol use.[321] Canada has one of the highest rates of adult obesity among Organisation for Economic Co-operation and Development (OECD) countries, contributing to approximately 2.7 million cases of diabetes (types 1 and 2 combined).[321] Four chronic diseases—cancer (leading cause of death), cardiovascular diseases, respiratory diseases and diabetes—account for 65 percent of deaths in Canada.[322][323]\n\nBased on the prior context, Has Canada experienced any famines?\nYes or no answer:"
1,Japan,http://www.wikidata.org/entity/Q17,Q17,Japan,https://en.wikipedia.org/wiki/Japan,DeferredCell,"\nJapan spent 10.74% of its total GDP on healthcare in 2019.[267] In 2020, the overall life expectancy in Japan at birth was 84.62 years (81.64 years for males and 87.74 years for females), the highest in the world;[268] while it had a very low infant mortality rate (2 per 1,000 live births).[269] Since 1981, the principal cause of death in Japan is cancer, which accounted for 27% of the total deaths in 2018—followed by cardiovascular diseases, which led to 15% of the deaths.[270] Japan has one of the world's highest suicide rates, which is considered a major social issue.[271] Another significant public health issue is smoking among Japanese men.[272] However, Japan has the lowest rate of heart disease in the OECD, and the lowest level of dementia among developed countries.[273]\n\nJapan is substantially prone to earthquakes, tsunami and volcanic eruptions because of its location along the Pacific Ring of Fire.[85] It has the 17th highest natural disaster risk as measured in the 2016 World Risk Index.[86] Japan has 111 active volcanoes.[87] Destructive earthquakes, often resulting in tsunami, occur several times each century;[88] the 1923 Tokyo earthquake killed over 140,000 people.[89] More recent major quakes are the 1995 Great Hanshin earthquake and the 2011 Tōhoku earthquake, which triggered a large tsunami.[70]\n\nBased on the prior context, Has Japan experienced any famines?\nYes or no answer:"
2,Norway,http://www.wikidata.org/entity/Q20,Q20,Norway,https://en.wikipedia.org/wiki/Norway,DeferredCell,"\nThe famine of 1695–1696 killed roughly 10% of Norway's population.[80] The harvest failed in Scandinavia at least nine times between 1740 and 1800, with great loss of life.[81]\n\nIn 1349, the Black Death radically altered Norway, killing between 50% and 60% of its population[75] and leaving it in a

In [18]:
df["prompt"][2]

"\nThe famine of 1695–1696 killed roughly 10% of Norway's population.[80] The harvest failed in Scandinavia at least nine times between 1740 and 1800, with great loss of life.[81]\n\nIn 1349, the Black Death radically altered Norway, killing between 50% and 60% of its population[75] and leaving it in a period of social and economic decline.[76] The plague left Norway very poor. Although the death rate was comparable with the rest of Europe, economic recovery took much longer because of the small, scattered population.[76] Even before the plague, the population was only about 500,000.[77] After the plague, many farms lay idle while the population slowly increased.[76] However, the few surviving farms' tenants found their bargaining positions with their landlords greatly strengthened.[76]\n\nBased on the prior context, Has Norway experienced any famines?\nYes or no answer:"

In [36]:
print(examples + df["prompt"][2])


In 2011, Canadian forces participated in the NATO-led intervention into the Libyan Civil War,[118] and also became involved in battling the Islamic State insurgency in Iraq in the mid-2010s.[119] The COVID-19 pandemic in Canada began on January 27, 2020, with wide social and economic disruption.[120] In 2021, the remains of hundreds of Indigenous people were discovered near the former sites of Canadian Indian residential schools.[121] Administered by the Canadian Catholic Church and funded by the Canadian government from 1828 to 1997, these boarding schools attempted to assimilate Indigenous children into Euro-Canadian culture.[122]

In common with many other developed countries, Canada is experiencing an increase in healthcare expenditures due to a demographic shift toward an older population, with more retirees and fewer people of working age. In 2006, the average age in Canada was 39.5 years;[317] it rose to 42.4 years by 2018[318] before falling slightly to 41.9 in 2021.[309] Life

In [21]:
df["prompt"][2]

"\nThe famine of 1695–1696 killed roughly 10% of Norway's population.[80] The harvest failed in Scandinavia at least nine times between 1740 and 1800, with great loss of life.[81]\n\nIn 1349, the Black Death radically altered Norway, killing between 50% and 60% of its population[75] and leaving it in a period of social and economic decline.[76] The plague left Norway very poor. Although the death rate was comparable with the rest of Europe, economic recovery took much longer because of the small, scattered population.[76] Even before the plague, the population was only about 500,000.[77] After the plague, many farms lay idle while the population slowly increased.[76] However, the few surviving farms' tenants found their bargaining positions with their landlords greatly strengthened.[76]\n\nBased on the prior context, Has Norway experienced any famines?\nYes or no answer:"

In [35]:
question_emb = encode_question(["Is the name of the country based on an indigenous language?"])

In [36]:
mk.search(data=ctx_df, query=question_emb, by="ctx_emb", k=5)

,paragraph,section,title,ctx_emb
0,"The country's ten largest self-reported specific ethnic or cultural origins in 2021 were Canadian[c] (accounting for 15.6 percent of the population), followed by English (14.7 percent), Irish (12.1 percent), Scottish (12.1 percent), French (11.0 percent), German (8.1 percent), Chinese (4.7 percent), Italian (4.3 percent), Indian (3.7 percent), and Ukrainian (3.5 percent).[354]\n",Ethnicity[edit],Canada,torch.Tensor(shape=torch.Size([768]))
1,"Indigenous peoples in present-day Canada include the First Nations, Inuit, and Métis,[20] the last being of mixed descent who originated in the mid-17th century when First Nations people married European settlers and subsequently developed their own identity.[20]\n",Indigenous peoples[edit],Canada,torch.Tensor(shape=torch.Size([768]))
2,"Canada's culture draws influences from its broad range of constituent nationalities, and policies that promote a ""just society"" are constitutionally protected.[395][396][397] Canada has placed emphasis on equality and inclusiveness for all its people.[398][399] The official state policy of multiculturalism is often cited as one of Canada's significant accomplishments,[400] and a key distinguishing element of Canadian identity.[401][402] In Quebec, cultural identity is strong, and there is a French Canadian culture that is distinct from English Canadian culture.[403] As a whole, Canada is in theory a cultural mosaic of regional ethnic subcultures.[404][405]\n",Culture[edit],Canada,torch.Tensor(shape=torch.Size([768]))
3,"Over time, indigenous cultures in North America grew increasingly sophisticated, and some, such as the pre-Columbian Mississippian culture in the southeast, developed advanced agriculture, architecture, and complex societies.[43] The city-state of Cahokia is the largest, most complex pre-Columbian archaeological site in the modern-day United States.[44] In the Four Corners region, Ancestral Puebloan culture developed from centuries of agricultural experimentation.[45] The Algonquian are one of the most populous and widespread North American native language groups. Historically, the peoples were prominent along the Atlantic Coast and into the interior along the Saint Lawrence River and around the Great Lakes. This grouping consists of the peoples who speak Algonquian languages.[46] Before Europeans came into contact, most Algonquian settlements lived by hunting and fishing, although quite a few supplemented their diet by cultivating corn, beans and squash (the ""Three Sisters""). The Ojibwe cultivated wild rice.[47] The Haudenosaunee of the Iroquois, located in the southern Great Lakes region, was established at some point between the twelfth and fifteenth centuries.[48]\n",Indigenous peoples and pre-Columbian history[edit],United States,torch.Tensor(shape=torch.Size([768]))
4,"There are older historical divisions, primarily the twenty-five provinces and three lands, which still retain cultural significance.\n",Administrative divisions[edit],Sweden,torch.Tensor(shape=torch.Size([768]))


In [65]:
import torch
df["score"] = torch.matmul(df["ctx_emb"], question_emb.T).squeeze()
df.sort(by="score", ascending=False)

  df["score"] = torch.matmul(df["ctx_emb"], question_emb.T).squeeze()

[2023-01-15 19:06:36,012][WARNING][py.warnings:109] :: /var/folders/0g/9gpzpkwn2j74ntfk90_9t31m0000gp/T/ipykernel_27273/3391258669.py:2: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484611838/work/torch/csrc/utils/python_arg_parser.cpp:296.)
  df["score"] = torch.matmul(df["ctx_emb"], question_emb.T).squeeze()



,paragraph,section,ctx_emb,score
0,"Indigenous peoples in present-day Canada include the First Nations, Inuit, and Métis,[20] the last being of mixed descent who originated in the mid-17th century when First Nations people married European settlers and subsequently developed their own identity.[20]\n",Indigenous peoples[edit],torch.Tensor(shape=torch.Size([768])),tensor(69.5354)
1,"While a variety of theories have been postulated for the etymological origins of Canada, the name is now accepted as coming from the St. Lawrence Iroquoian word kanata, meaning ""village"" or ""settlement"".[9] In 1535, Indigenous inhabitants of the present-day Quebec City region used the word to direct French explorer Jacques Cartier to the village of Stadacona.[10] Cartier later used the word Canada to refer not only to that particular village but to the entire area subject to Donnacona (the chief at Stadacona);[10] by 1545, European books and maps had begun referring to this small region along the Saint Lawrence River as Canada.[10]\n",Etymology[edit],torch.Tensor(shape=torch.Size([768])),tensor(68.7633)
2,"The Indigenous population at the time of the first European settlements is estimated to have been between 200,000[27] and two million,[28] with a figure of 500,000 accepted by Canada's Royal Commission on Aboriginal Peoples.[29] As a consequence of European colonization, the Indigenous population declined by forty to eighty percent, and several First Nations, such as the Beothuk, disappeared.[30] The decline is attributed to several causes, including the transfer of European diseases, such as influenza, measles, and smallpox to which they had no natural immunity,[27][31] conflicts over the fur trade, conflicts with the colonial authorities and settlers, and the loss of Indigenous lands to settlers and the subsequent collapse of several nations' self-sufficiency.[32][33]\n",Indigenous peoples[edit],torch.Tensor(shape=torch.Size([768])),tensor(68.3305)
3,"According to the 2021 Canadian census, over 450 ""ethnic or cultural origins"" were self-reported by Canadians.[349] The major panethnic groups chosen were: European (52.5 percent), North American (22.9 percent), Asian (19.3 percent), North American Indigenous (6.1 percent), African (3.8 percent), Latin, Central and South American (2.5 percent), Caribbean (2.1 percent), Oceanian (0.3 percent), and other (6 percent).[349][350] Statistics Canada reports that 35.5 percent of the population reported multiple ethnic origins, thus the overall total is greater than 100 percent.[349]\n",Ethnicity[edit],torch.Tensor(shape=torch.Size([768])),tensor(67.5559)
4,"The country's ten largest self-reported specific ethnic or cultural origins in 2021 were Canadian[c] (accounting for 15.6 percent of the population), followed by English (14.7 percent), Irish (12.1 percent), Scottish (12.1 percent), French (11.0 percent), German (8.1 percent), Chinese (4.7 percent), Italian (4.3 percent), Indian (3.7 percent), and Ukrainian (3.5 percent).[354]\n",Ethnicity[edit],torch.Tensor(shape=torch.Size([768])),tensor(66.1394)
...,...,...,...,...
101,Overviews\n,External links[edit],torch.Tensor(shape=torch.Size([768])),tensor(48.2864)
102,Government\n,External links[edit],torch.Tensor(shape=torch.Size([768])),tensor(47.4628)
103,"Canada's judiciary plays an important role in interpreting laws and has the power to strike down Acts of Parliament that violate the constitution. The Supreme Court of Canada is the highest court and final arbiter and has been led since December 18, 2017, by Richard Wagner, the chief justice of Canada.[205] The governor general appoints its nine members on the advice of the prime minister and minister of justice. All judges at the superior and appellate levels are appointed after consultation with non-governmental legal bodies. The federal Cabinet also appoints justices to superior courts in the provincial and territorial jurisdictions.[206]\n",Law[edit],torch.Tensor(shape=torch.Size([768])),tensor(4

In [ ]:
df.nnsearch(by="")

In [32]:
section

In [16]:
# Extract the headers
soup = BeautifulSoup(df["html"][0]()["parse"]["text"]["*"])
headers = [header.get_text() for header in soup.find_all(["h1","h2","h3","h4","h5","h6"])]

In [22]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [21]:

input_ids = tokenizer("Hello, is my dog cute ?", return_tensors="pt")["input_ids"]
embeddings = model(input_ids).pooler_output


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
df

,country,uri,qid,title,wikipedia_url,html
0,Canada,http://www.wikidata.org/entity/Q16,Q16,Canada,https://en.wikipedia.org/wiki/Canada,DeferredCell
1,Japan,http://www.wikidata.org/entity/Q17,Q17,Japan,https://en.wikipedia.org/wiki/Japan,DeferredCell
2,Norway,http://www.wikidata.org/entity/Q20,Q20,Norway,https://en.wikipedia.org/wiki/Norway,DeferredCell
3,Wales,http://www.wikidata.org/entity/Q25,Q25,Wales,https://en.wikipedia.org/wiki/Wales,DeferredCell
4,Republic of Ireland,http://www.wikidata.org/entity/Q27,Q27,Republic of Ireland,https://en.wikipedia.org/wiki/Republic_of_Ireland,DeferredCell
5,Hungary,http://www.wikidata.org/entity/Q28,Q28,Hungary,https://en.wikipedia.org/wiki/Hungary,DeferredCell
6,United States of America,http://www.wikidata.org/entity/Q30,Q30,United States,https://en.wikipedia.org/wiki/United_States,DeferredCell
7,Finland,http://www.wikidata.org/entity/Q33,Q33,Finland,https://en.wikipedia.org/wiki/Finland,DeferredCell
8,Sweden,http://www.wikidata.org/entity/Q34,Q34,Sweden,https://en.wikipedia.org/wiki/Sweden,DeferredCell
9,Denmark,http://www.wikidata.org/entity/Q35,Q35,Denmark,https://en.wikipedia.org/wiki/Denmark,DeferredCell


In [18]:
# Extract the headers
headers = soup.find_all(["h1","h2","h3","h4","h5","h6"])

# Iterate through the headers
for header in headers:
    level = int(header.name[1])
    indentation = "    " * (level-1)
    print(f"{indentation}{header.get_text()}")

    Contents
    Etymology[edit]
    History[edit]
        Indigenous peoples[edit]
        European colonization[edit]
        British North America[edit]
        Confederation and expansion[edit]
        Early 20th century[edit]
        Contemporary era[edit]
    Geography[edit]
        Climate[edit]
        Biodiversity[edit]
    Government and politics[edit]
        Law[edit]
        Foreign relations and military[edit]
        Provinces and territories[edit]
    Economy[edit]
        Science and technology[edit]
    Demographics[edit]
        Health[edit]
        Education[edit]
        Ethnicity[edit]
        Languages[edit]
        Religion[edit]
    Culture[edit]
        Symbols[edit]
        Literature[edit]
        Media[edit]
        Visual arts[edit]
        Music[edit]
        Sports[edit]
    See also[edit]
    Notes[edit]
    References[edit]
    Further reading[edit]
    External links[edit]


In [14]:
from bs4 import 
print(BeautifulSoup(df["html"][0]()["parse"]["text"]["*"]).get_text())

Country in North America
For other uses, see Canada (disambiguation).

Coordinates: 60°N 110°W﻿ / ﻿60°N 110°W﻿ / 60; -110

Canada


Flag



Coat of arms

Motto: A mari usque ad mare (Latin)"From Sea to Sea"Anthem: "O Canada"Royal anthem: "God Save the King"[1]CapitalOttawa45°24′N 75°40′W﻿ / ﻿45.400°N 75.667°W﻿ / 45.400; -75.667Largest cityTorontoOfficial languagesEnglishFrenchEthnic groups See belowReligion See belowDemonym(s)CanadianGovernmentFederal parliamentaryconstitutional monarchy• Monarch Charles III• Governor General Mary Simon• Prime Minister Justin Trudeau
LegislatureParliament• Upper houseSenate• Lower houseHouse of CommonsIndependence from the United Kingdom• Confederation July 1, 1867• Statute of Westminster 1931 December 11, 1931• Patriation April 17, 1982
Area • Total area9,984,670 km2 (3,855,100 sq mi) (2nd)• Water (%)11.76 (as of 2015)[2]• Total land area9,093,507 km2 (3,511,023 sq mi)Population• Q4 2022 estimate39,292,355[3] (37th)• 2021 census36,991,981[4]• Density4

In [7]:
df.gui.gallery(main_column="wikipedia_url", tag_columns=["title"])

In [24]:
df

,country,uri,qid,title,wikipedia_url
0,Canada,http://www.wikidata.org/entity/Q16,Q16,Canada,https://en.wikipedia.org/wiki/Canada
1,Japan,http://www.wikidata.org/entity/Q17,Q17,Japan,https://en.wikipedia.org/wiki/Japan
2,Norway,http://www.wikidata.org/entity/Q20,Q20,Norway,https://en.wikipedia.org/wiki/Norway
3,Wales,http://www.wikidata.org/entity/Q25,Q25,Wales,https://en.wikipedia.org/wiki/Wales
4,Republic of Ireland,http://www.wikidata.org/entity/Q27,Q27,Republic of Ireland,https://en.wikipedia.org/wiki/Republic_of_Ireland
5,Hungary,http://www.wikidata.org/entity/Q28,Q28,Hungary,https://en.wikipedia.org/wiki/Hungary
6,United States of America,http://www.wikidata.org/entity/Q30,Q30,United States,https://en.wikipedia.org/wiki/United_States
7,Finland,http://www.wikidata.org/entity/Q33,Q33,Finland,https://en.wikipedia.org/wiki/Finland
8,Sweden,http://www.wikidata.org/entity/Q34,Q34,Sweden,https://en.wikipedia.org/wiki/Sweden
9,Denmark,http://www.wikidata.org/entity/Q35,Q35,Denmark,https://en.wikipedia.org/wiki/Denmark
